In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')
folder_path = '/content/gdrive/My Drive/Data 144 Kaggle Challenge/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train = pd.read_csv(folder_path+'yelp_train.csv')
test = pd.read_csv(folder_path+'yelp_test.csv')

sample = pd.read_csv(folder_path+'yelp_sample.csv')

In [0]:
train.columns

Index(['review_id', 'business_id', 'user_id', 'date', 'cool', 'funny',
       'useful', 'text', 'user_average_stars', 'user_elite',
       'user_review_count', 'user_yelping_since', 'business_categories',
       'business_city', 'business_latitude', 'business_longitude',
       'business_review_count', 'business_average_stars', 'business_state',
       'is_good_rating'],
      dtype='object')

In [0]:
train.is_good_rating.head()

0    0
1    0
2    1
3    0
4    0
Name: is_good_rating, dtype: int64

In [0]:
train.text.head()

0    Pool tables, darts, and other neighborhood bar...
1    This place was alright. However, their classic...
2    First time at Pieology Pizzeria and I was impr...
3    Its a good facility with lots of equipment and...
4    Very very disappointed! My shawarma chicken wa...
Name: text, dtype: object

In [0]:
train.is_good_rating.value_counts()

1    159858
0     80142
Name: is_good_rating, dtype: int64

In [0]:
train.business_categories

0         Restaurants, Sports Bars, Bars, American (Trad...
1                                Beauty & Spas, Nail Salons
2                                        Pizza, Restaurants
3         Active Life, Fitness & Instruction, Trainers, ...
4                                Mediterranean, Restaurants
5                            Optometrists, Health & Medical
6         Hotels, Resorts, Casinos, Event Planning & Ser...
7         Breakfast & Brunch, Diners, American (New), Re...
8                              Pest Control, Local Services
9         Automotive, Auto Parts & Supplies, Tires, Whee...
10                              Cuban, Restaurants, Mexican
11                          Food, Ice Cream & Frozen Yogurt
12        American (Traditional), Restaurants, Beer Bar,...
13        Bars, Pubs, Food, Cocktail Bars, Nightlife, Ci...
14        Desserts, American (New), Sandwiches, American...
15                                     Mexican, Restaurants
16        American (Traditional), Americ

In [0]:
train.groupby('business_categories').size()

business_categories
3D Printing, Computers, Shopping, Printing Services, Local Services, Electronics                                                                                          1
3D Printing, Printing Services, Local Services                                                                                                                            1
ATV Rentals/Tours, Bike Rentals, Bikes, Active Life, Hotels & Travel, Parks, Sporting Goods, Tours, Shopping, Recreation Centers                                          1
ATV Rentals/Tours, Tours, Active Life, Hotels & Travel                                                                                                                    1
Acai Bowls, Ice Cream & Frozen Yogurt, Juice Bars & Smoothies, Gluten-Free, Food, Restaurants                                                                             2
Acai Bowls, Juice Bars & Smoothies, Ice Cream & Frozen Yogurt, Food                                                     

In [0]:
train['bizcategory1'] = train.business_categories.apply(lambda x: x.split(', ')[0] if type(x)==str else '---')
train['bizcategory2'] = ['---' if type(i)!=str else '---' if len(i.split(', '))==1 else i.split(', ')[1] for i in train.business_categories]
train['top2categories'] = train['bizcategory1'].str.cat(train['bizcategory2'], sep=' ')
top_categories = train.top2categories.value_counts()
top_categories = [k for k,v in top_categories.iteritems() if v > 100]

In [0]:
len(train.top2categories.unique())

11028

In [0]:
len(train.bizcategory1.unique())

1008

In [0]:
len(train.bizcategory2.unique())

1016

In [0]:
print(top_categories)

['Mexican Restaurants', 'Restaurants Mexican', 'Italian Restaurants', 'American (New) Restaurants', 'Pizza Restaurants', 'Restaurants Pizza', 'Restaurants Italian', 'Chinese Restaurants', 'Restaurants Chinese', 'Restaurants American (New)', 'Restaurants American (Traditional)', 'Breakfast & Brunch Restaurants', 'Restaurants Breakfast & Brunch', 'American (Traditional) Restaurants', 'Bars Nightlife', 'Nightlife Bars', 'Restaurants Burgers', 'Coffee & Tea Food', 'Food Coffee & Tea', 'Restaurants Food', 'Japanese Restaurants', 'Restaurants Japanese', 'Restaurants Bars', 'Restaurants Nightlife', 'Restaurants Sandwiches', 'Restaurants Thai', 'Sandwiches Restaurants', 'Restaurants Sushi Bars', 'Burgers Restaurants', 'Steakhouses Restaurants', 'Sushi Bars Restaurants', 'Food Restaurants', 'Restaurants Steakhouses', 'Nail Salons Beauty & Spas', 'Thai Restaurants', 'Beauty & Spas Nail Salons', 'Restaurants Seafood', 'Bars Restaurants', 'Restaurants Vietnamese', 'Nightlife Restaurants', 'Hotels 

In [0]:
print(len(top_categories))

399


In [0]:
# print(train.business_latitude.str.cat(train.business_longitude, sep=' '))
train.business_latitude.head()

0    33.336050
1    35.058040
2    36.114884
3    36.025332
4    36.005901
Name: business_latitude, dtype: float64

In [0]:
train.business_longitude.head()

0   -111.844343
1    -80.937064
2   -115.155604
3   -115.120852
4   -115.111863
Name: business_longitude, dtype: float64

In [0]:
# looked up the midpoint of the united states
middle_lat = 39.8333333
middle_lon = -98.585522

# compare with longitude latitude of the business to classify as northwest, northeast, southwest, southeast
def find_region(business_lat, business_lon):
  if np.isnan(business_lat) or np.isnan(business_lon):
    return np.nan
  if business_lat > middle_lat:
    if business_lon > middle_lon:
      return "North East"
    return "North West"
  if business_lon > middle_lon:
    return "South East"
  return "South West"
business_region = [find_region(x,y) for x,y in zip(train.business_latitude.values, train.business_longitude.values)]

In [0]:
business_region[:10]

['South West',
 'South East',
 'South West',
 'South West',
 'South West',
 'South West',
 'South West',
 'North East',
 'South West',
 'South West']

In [0]:
train.business_state[:10]

0    AZ
1    SC
2    NV
3    NV
4    NV
5    AZ
6    NV
7    ON
8    NV
9    AZ
Name: business_state, dtype: object

In [0]:
train.business_state.value_counts()

NV     83236
AZ     75484
ON     27832
NC     14146
OH     12288
PA     10034
QC      6148
WI      4968
EDH     2255
IL      1419
BW      1361
SC       450
HLD       99
MLN       98
CHE       34
NYK       33
FL        28
FIF       19
ESX       12
WLN       12
ELN        7
C          7
NY         6
GLG        4
NI         3
XGL        3
01         2
AR         2
ABE        2
AK         2
CA         1
FAL        1
TAM        1
B          1
IN         1
KHL        1
Name: business_state, dtype: int64

In [0]:
# state codes are not unified, also this data seems to be coming from a bunch of different places, not just the united states

In [0]:
train.business_city[:10]

0           Chandler
1          Fort Mill
2          Las Vegas
3          Las Vegas
4          Henderson
5           Chandler
6          Las Vegas
7            Toronto
8    North Las Vegas
9           Chandler
Name: business_city, dtype: object

In [0]:
# judging from the Toronto, ON entry, there are canadian businesses in this data???
train['business_region'] = business_region

In [0]:
train.groupby(['business_region', 'is_good_rating']).size()

business_region  is_good_rating
North East       0                  23739
                 1                  42871
North West       1                      1
South East       0                   4978
                 1                   9685
South West       0                  51424
                 1                 107300
dtype: int64

In [0]:
# clearly some of the regions are incorrect, AZ should not be in the north west
train[train.business_region=='North West']

,review_id,business_id,user_id,date,cool,funny,useful,text,user_average_stars,user_elite,user_review_count,user_yelping_since,business_categories,business_city,business_latitude,business_longitude,business_review_count,business_average_stars,business_state,is_good_rating,bizcategory1,bizcategory2,top2categories,business_region,user_days_on_yelp,user_reviews_per_day,user_times_elite,authored_by_elite_user,user_business_avg_star_difference,business_total_stars
182361,sbYPqcFihGOpZYZdfjLMng,0MAubmdZUjeCHlqy5nMuDw,VkyZgMTwfr3RalQLvKglLQ,2016-02-09,0,0,0,I had Ray repair 4 different spots that my dog...,4.2,None,5,2014-01-29,"Flooring, Carpet Cleaning, Carpet Installation...",Mesa,47.308661,-122.397206,14,5.0,AZ,1,Flooring,Carpet Cleaning,Flooring Carpet Cleaning,North West,2072,0.002413,0,0,-0.8,70.0


In [0]:
# this probably indicates that we should not use business_region or business_state as a feature
# might need to think of other features that are more consistent or informative

In [0]:
train['business_total_stars'] = train.business_review_count * train.business_average_stars

In [0]:
user_stats = ['user_average_stars', 'user_elite', 'user_review_count', 'user_yelping_since']
print(train.user_elite.head())

0          None
1          None
2          None
3    2013, 2012
4          None
Name: user_elite, dtype: object


In [0]:
elite_count = train.user_elite.apply(lambda x: 0 if x=='None' else len(x.split(', ')))
elite_count.head()

0    0
1    0
2    0
3    2
4    0
Name: user_elite, dtype: int64

In [0]:
elite_count.max()

12

In [0]:
train.user_yelping_since.head()

0    2013-10-23
1    2016-07-04
2    2013-09-05
3    2010-09-03
4    2016-03-24
Name: user_yelping_since, dtype: object

In [0]:
user_yelping_since = pd.to_datetime(train.user_yelping_since)
user_yelping_since.head()

0   2013-10-23
1   2016-07-04
2   2013-09-05
3   2010-09-03
4   2016-03-24
Name: user_yelping_since, dtype: datetime64[ns]

In [0]:
train.user_review_count.head()

0     5
1     7
2    21
3    83
4    28
Name: user_review_count, dtype: int64

In [0]:
# some reviewers have not been "active" -- very few reviews since they started yelping, whereas others have been active or elite
user_days_on_yelp = (pd.datetime.now()-user_yelping_since).dt.days
user_days_on_yelp.head()

0    2170
1    1185
2    2218
3    3316
4    1287
Name: user_yelping_since, dtype: int64

In [0]:
user_reviews_per_day = train.user_review_count / user_days_on_yelp
user_reviews_per_day.head()

0    0.002304
1    0.005907
2    0.009468
3    0.025030
4    0.021756
dtype: float64

In [0]:
train['user_days_on_yelp'] = user_days_on_yelp
train['user_yelping_since'] = user_yelping_since
train['user_reviews_per_day'] = user_reviews_per_day
train['user_times_elite'] = elite_count
train.head()

,review_id,business_id,user_id,date,cool,funny,useful,text,user_average_stars,user_elite,user_review_count,user_yelping_since,business_categories,business_city,business_latitude,business_longitude,business_review_count,business_average_stars,business_state,is_good_rating,bizcategory1,bizcategory2,top2categories,business_region,user_days_on_yelp,user_reviews_per_day,user_times_elite
0,9uu-JXdQuQ3rXOd1Vxv8kA,8zN3nV0zbtE377_XhlvabA,MWahC54Rd_CNv_0VtjCD6A,2017-06-27,0,0,0,"Pool tables, darts, and other neighborhood bar...",2.00,None,5,2013-10-23,"Restaurants, Sports Bars, Bars, American (Trad...",Chandler,33.336050,-111.844343,158,4.0,AZ,0,Restaurants,Sports Bars,Restaurants Sports Bars,South West,2170,0.002304,0
1,UbCgugoTK2MfBmtU_8PXaQ,wJ-PcjemwR44WitIJsDzsA,N-GOto4b3ltfGJ7y-5g-pg,2017-03-25,0,0,0,"This place was alright. However, their classic...",4.43,None,7,2016-07-04,"Beauty & Spas, Nail Salons",Fort Mill,35.058040,-80.937064,26,3.5,SC,0,Beauty & Spas,Nail Salons,Beauty & Spas Nail Salons,South East,1185,0.005907,0
2,6oF9_NR1O5Y_NOE6INXfrg,fP4sMlVw_MvyGNv4xeTGDA,RH3w77UyXaQPiOx2jOE7zg,2017-07-14,0,0,0,First time at Pieology Pizzeria and I was impr...,4.09,None,21,2013-09-05,"Pizza, Restaurants",Las Vegas,36.114884,-115.155604,189,4.0,NV,1,Pizza,Restaurants,Pizza Restaurants,South West,2218,0.009468,0
3,kyYWrcrr83GnJ7m_VEvwCA,hgWMxKhrnOUd3m5nOUBIkA,g6pwf1E-CylnZQRoaZGGdw,2010-12-13,1,0,0,Its a good facility with lots of equipment and...,3.55,"2013, 2012",83,2010-09-03,"Active Life, Fitness & Instruction, Trainers, ...",Las Vegas,36.025332,-115.120852,316,3.0,NV,0,Active Life,Fitness & Instruction,Active Life Fitness & Instruction,South West,3316,0.025030,2
4,K2X3XG9_M3EM0TWguo2bCw,KMKPusWbBaIORB669W0EeQ,14nl002my6qLaEcqYHLPig,2016-09-12,1,1,1,Very very disappointed! My shawarma chicken wa...,3.75,None,28,2016-03-24,"Mediterranean, Restaurants",Henderson,36.005901,-115.111863,61,3.5,NV,0,Mediterranean,Restaurants,Mediterranean Restaurants,South West,1287,0.021756,0


In [0]:
train.date = pd.to_datetime(train.date)
train.date.head()

0   2017-06-27
1   2017-03-25
2   2017-07-14
3   2010-12-13
4   2016-09-12
Name: date, dtype: datetime64[ns]

In [0]:
authored_by_elite_user = [1 if x in y else 0 for x,y in zip(train.date.dt.strftime('%Y'),train.user_elite)]
authored_by_elite_user = pd.Series(authored_by_elite_user)
authored_by_elite_user.value_counts()

0    194072
1     45928
dtype: int64

In [0]:
train['authored_by_elite_user'] = authored_by_elite_user

In [0]:
train.user_average_stars.head()

0    2.00
1    4.43
2    4.09
3    3.55
4    3.75
Name: user_average_stars, dtype: float64

In [0]:
train.user_average_stars.describe()

count    240000.000000
mean          3.750624
std           0.794099
min           1.000000
25%           3.420000
50%           3.820000
75%           4.200000
max           5.000000
Name: user_average_stars, dtype: float64

In [0]:
user_likely_critical = train.user_average_stars-train.business_average_stars
user_likely_critical.head()

0   -2.00
1    0.93
2    0.09
3    0.55
4    0.25
dtype: float64

In [0]:
train['user_likely_critical'] = user_likely_critical
train.rename({'user_likely_critical': 'user_business_avg_star_difference'}, axis=1, inplace=True)
train.head()

,review_id,business_id,user_id,date,cool,funny,useful,text,user_average_stars,user_elite,user_review_count,user_yelping_since,business_categories,business_city,business_latitude,business_longitude,business_review_count,business_average_stars,business_state,is_good_rating,bizcategory1,bizcategory2,top2categories,business_region,user_days_on_yelp,user_reviews_per_day,user_times_elite,authored_by_elite_user,user_business_avg_star_difference
0,9uu-JXdQuQ3rXOd1Vxv8kA,8zN3nV0zbtE377_XhlvabA,MWahC54Rd_CNv_0VtjCD6A,2017-06-27,0,0,0,"Pool tables, darts, and other neighborhood bar...",2.00,None,5,2013-10-23,"Restaurants, Sports Bars, Bars, American (Trad...",Chandler,33.336050,-111.844343,158,4.0,AZ,0,Restaurants,Sports Bars,Restaurants Sports Bars,South West,2170,0.002304,0,0,-2.00
1,UbCgugoTK2MfBmtU_8PXaQ,wJ-PcjemwR44WitIJsDzsA,N-GOto4b3ltfGJ7y-5g-pg,2017-03-25,0,0,0,"This place was alright. However, their classic...",4.43,None,7,2016-07-04,"Beauty & Spas, Nail Salons",Fort Mill,35.058040,-80.937064,26,3.5,SC,0,Beauty & Spas,Nail Salons,Beauty & Spas Nail Salons,South East,1185,0.005907,0,0,0.93
2,6oF9_NR1O5Y_NOE6INXfrg,fP4sMlVw_MvyGNv4xeTGDA,RH3w77UyXaQPiOx2jOE7zg,2017-07-14,0,0,0,First time at Pieology Pizzeria and I was impr...,4.09,None,21,2013-09-05,"Pizza, Restaurants",Las Vegas,36.114884,-115.155604,189,4.0,NV,1,Pizza,Restaurants,Pizza Restaurants,South West,2218,0.009468,0,0,0.09
3,kyYWrcrr83GnJ7m_VEvwCA,hgWMxKhrnOUd3m5nOUBIkA,g6pwf1E-CylnZQRoaZGGdw,2010-12-13,1,0,0,Its a good facility with lots of equipment and...,3.55,"2013, 2012",83,2010-09-03,"Active Life, Fitness & Instruction, Trainers, ...",Las Vegas,36.025332,-115.120852,316,3.0,NV,0,Active Life,Fitness & Instruction,Active Life Fitness & Instruction,South West,3316,0.025030,2,0,0.55
4,K2X3XG9_M3EM0TWguo2bCw,KMKPusWbBaIORB669W0EeQ,14nl002my6qLaEcqYHLPig,2016-09-12,1,1,1,Very very disappointed! My shawarma chicken wa...,3.75,None,28,2016-03-24,"Mediterranean, Restaurants",Henderson,36.005901,-115.111863,61,3.5,NV,0,Mediterranean,Restaurants,Mediterranean Restaurants,South West,1287,0.021756,0,0,0.25


In [0]:
# eda on review type
print(train[train.is_good_rating==1][['cool','funny','useful']].describe())
print('good reviews ^^ ')
print(train[train.is_good_rating==0][['cool','funny','useful']].describe())
print('bad reviews ^^')

                cool          funny         useful
count  159858.000000  159858.000000  159858.000000
mean        0.613132       0.382258       1.134976
std         2.897990       2.624454       3.363687
min         0.000000       0.000000       0.000000
25%         0.000000       0.000000       0.000000
50%         0.000000       0.000000       0.000000
75%         1.000000       0.000000       1.000000
max       234.000000     589.000000     246.000000
good reviews ^^ 
               cool         funny        useful
count  80142.000000  80142.000000  80142.000000
mean       0.407377      0.624379      1.724165
std        3.023209      3.473608      8.507285
min        0.000000      0.000000      0.000000
25%        0.000000      0.000000      0.000000
50%        0.000000      0.000000      1.000000
75%        0.000000      1.000000      2.000000
max      245.000000    447.000000   1456.000000
bad reviews ^^


In [0]:
# seems like useful reviews tend to be strong indicators of a bad/negative rating
# cool reviews tend to be slightly indicative of a good rating 
# funny reviews don't tend to inform the decision (at least according to this training set)

In [0]:
train.head()

,review_id,business_id,user_id,date,cool,funny,useful,text,user_average_stars,user_elite,user_review_count,user_yelping_since,business_categories,business_city,business_latitude,business_longitude,business_review_count,business_average_stars,business_state,is_good_rating,bizcategory1,bizcategory2,top2categories,business_region,user_days_on_yelp,user_reviews_per_day,user_times_elite,authored_by_elite_user,user_business_avg_star_difference,business_total_stars
0,9uu-JXdQuQ3rXOd1Vxv8kA,8zN3nV0zbtE377_XhlvabA,MWahC54Rd_CNv_0VtjCD6A,2017-06-27,0,0,0,"Pool tables, darts, and other neighborhood bar...",2.00,None,5,2013-10-23,"Restaurants, Sports Bars, Bars, American (Trad...",Chandler,33.336050,-111.844343,158,4.0,AZ,0,Restaurants,Sports Bars,Restaurants Sports Bars,South West,2170,0.002304,0,0,-2.00,632.0
1,UbCgugoTK2MfBmtU_8PXaQ,wJ-PcjemwR44WitIJsDzsA,N-GOto4b3ltfGJ7y-5g-pg,2017-03-25,0,0,0,"This place was alright. However, their classic...",4.43,None,7,2016-07-04,"Beauty & Spas, Nail Salons",Fort Mill,35.058040,-80.937064,26,3.5,SC,0,Beauty & Spas,Nail Salons,Beauty & Spas Nail Salons,South East,1185,0.005907,0,0,0.93,91.0
2,6oF9_NR1O5Y_NOE6INXfrg,fP4sMlVw_MvyGNv4xeTGDA,RH3w77UyXaQPiOx2jOE7zg,2017-07-14,0,0,0,First time at Pieology Pizzeria and I was impr...,4.09,None,21,2013-09-05,"Pizza, Restaurants",Las Vegas,36.114884,-115.155604,189,4.0,NV,1,Pizza,Restaurants,Pizza Restaurants,South West,2218,0.009468,0,0,0.09,756.0
3,kyYWrcrr83GnJ7m_VEvwCA,hgWMxKhrnOUd3m5nOUBIkA,g6pwf1E-CylnZQRoaZGGdw,2010-12-13,1,0,0,Its a good facility with lots of equipment and...,3.55,"2013, 2012",83,2010-09-03,"Active Life, Fitness & Instruction, Trainers, ...",Las Vegas,36.025332,-115.120852,316,3.0,NV,0,Active Life,Fitness & Instruction,Active Life Fitness & Instruction,South West,3316,0.025030,2,0,0.55,948.0
4,K2X3XG9_M3EM0TWguo2bCw,KMKPusWbBaIORB669W0EeQ,14nl002my6qLaEcqYHLPig,2016-09-12,1,1,1,Very very disappointed! My shawarma chicken wa...,3.75,None,28,2016-03-24,"Mediterranean, Restaurants",Henderson,36.005901,-115.111863,61,3.5,NV,0,Mediterranean,Restaurants,Mediterranean Restaurants,South West,1287,0.021756,0,0,0.25,213.5


In [0]:
good_reviews = train[train.is_good_rating==1]
bad_reviews = train[train.is_good_rating==0]

In [0]:
good_reviews.describe()

,cool,funny,useful,user_average_stars,user_review_count,business_latitude,business_longitude,business_review_count,business_average_stars,is_good_rating,user_days_on_yelp,user_reviews_per_day,user_times_elite,authored_by_elite_user,user_business_avg_star_difference,business_total_stars
count,159858.000000,159858.000000,159858.000000,159858.000000,159858.000000,159857.000000,159857.000000,159858.000000,159858.000000,159858.0,159858.000000,159858.000000,159858.000000,159858.000000,159858.000000,159858.000000
mean,0.613132,0.382258,1.134976,4.010405,109.536376,37.179890,-101.340383,355.556769,3.965297,1.0,2551.343461,0.036132,0.949242,0.195711,0.045107,1402.061029
std,2.897990,2.624454,3.363687,0.600613,238.983321,4.299659,20.561943,731.811817,0.615731,0.0,944.303676,0.068689,2.047782,0.396748,0.813381,2882.916704
min,0.000000,0.000000,0.000000,1.320000,0.000000,-36.086009,-122.499387,3.000000,1.000000,1.0,660.000000,0.000000,0.000000,0.000000,-3.680000,6.000000
25%,0.000000,0.000000,0.000000,3.650000,7.000000,33.587501,-115.154036,35.000000,3.500000,1.0,1830.000000,0.003410,0.000000,0.000000,-0.470000,136.000000
50%,0.000000,0.000000,0.000000,4.000000,24.000000,36.102251,-111.973506,117.000000,4.000000,1.0,2549.000000,0.010157,0.000000,0.000000,0.000000,452.000000
75%,1.000000,0.000000,1.000000,4.410000,100.000000,40.436993,-81.005802,342.000000,4.500000,1.0,3228.000000,0.036797,0.000000,0.000000,0.500000,1352.000000
max,234.000000,589.000000,246.000000,5.000000,3468.000000,59.438181,115.086769,7361.000000,5.000000,1.0,5468.000000,0.942135,12.000000,1.000000,4.000000,29444.000000


In [0]:
bad_reviews.describe()

,cool,funny,useful,user_average_stars,user_review_count,business_latitude,business_longitude,business_review_count,business_average_stars,is_good_rating,user_days_on_yelp,user_reviews_per_day,user_times_elite,authored_by_elite_user,user_business_avg_star_difference,business_total_stars
count,80142.000000,80142.000000,80142.000000,80142.000000,80142.000000,80141.000000,80141.000000,80142.000000,80142.000000,80142.0,80142.000000,80142.000000,80142.000000,80142.000000,80142.000000,80142.000000
mean,0.407377,0.624379,1.724165,3.232444,111.041888,37.410215,-100.538578,332.931210,3.288525,0.0,2564.837713,0.036346,0.920004,0.182701,-0.056081,1175.240885
std,3.023209,3.473608,8.507285,0.875075,243.161836,4.308290,20.339136,718.892901,0.764510,0.0,946.959650,0.070238,2.021163,0.386423,1.084344,2642.907697
min,0.000000,0.000000,0.000000,1.000000,0.000000,-34.510780,-122.499387,3.000000,1.000000,0.0,660.000000,0.000000,0.000000,0.000000,-4.000000,3.000000
25%,0.000000,0.000000,0.000000,2.930000,7.000000,33.626688,-115.154480,30.000000,3.000000,0.0,1844.000000,0.003194,0.000000,0.000000,-0.660000,90.000000
50%,0.000000,0.000000,1.000000,3.490000,23.000000,36.109538,-111.950366,96.000000,3.500000,0.0,2565.000000,0.009899,0.000000,0.000000,0.000000,315.000000
75%,0.000000,1.000000,2.000000,3.820000,103.000000,40.583216,-80.845023,290.000000,4.000000,0.0,3242.000000,0.037444,0.000000,0.000000,0.630000,1012.000000
max,245.000000,447.000000,1456.000000,4.940000,3468.000000,59.436505,115.086769,7361.000000,5.000000,0.0,5468.000000,0.942135,12.000000,1.000000,3.800000,29444.000000


In [0]:
# overall observations:

# - there are over 11000 unique values in top2categories 
# - bizcategory1 and bizcategory2 values are informative, but also have ~1000 unique values each
# - business_region, business_state are too noisy to use

# good and bad reviews don't really differ much based on: 
# - time on yelp or reviews per day
# - number of times author was elite



# good and bad reviews are different in that:
# - good reviews tend to come from elite users (in the years the users are elite)
# - bad reviews tend to have a negative user to business star difference (users visit restaurants that have a higher rating than what the users themselves typically give)
# - useful >>> negative, cool > positive, funny > negative
# - users giving negative reviews tend to have lower user_average_stars (typically more critical)

In [0]:
# trying it out with no text features, just user/business/voting features
def featurize_no_text(df):
  df['authored_by_elite_user'] = [1 if x in y else 0 for x,y in zip(pd.to_datetime(df.date).dt.strftime('%Y'), df.user_elite)]
  df['user_business_avg_star_difference'] = df.user_average_stars-df.business_average_stars
  return df[['authored_by_elite_user', 'useful', 'cool', 'funny', 'user_average_stars', 'user_business_avg_star_difference', 'business_average_stars']]

train_data = pd.read_csv(folder_path+'yelp_train.csv')
no_text_train = featurize_no_text(train_data)
no_text_train_y = train_data['is_good_rating']
no_text_test = featurize_no_text(pd.read_csv(folder_path+'yelp_test.csv'))

In [0]:
sample.head()

,review_id,is_good_rating
0,SiQGQ__poSeG1oKHVem30g,1
1,rFZPxXnZZuEfO5Cj8SprGw,1
2,DU9CeH0kUfN8LO7V02fxaA,1
3,dJgLik5M3vvHYHDkIW62qA,1
4,fi9Pbb8Z4bzzlHaxXGEJTg,1


In [0]:
test.head()

,review_id,business_id,user_id,date,cool,funny,useful,text,user_average_stars,user_elite,user_review_count,user_yelping_since,business_categories,business_city,business_latitude,business_longitude,business_review_count,business_average_stars,business_state
0,SiQGQ__poSeG1oKHVem30g,309bnXTHirKMU6wz1HwRjQ,oAil_gYOXUhqWnKA8SwKsw,2017-05-23,2,1,2,This was my first experience with Harry Buffal...,3.80,"2015, 2017, 2014, 2016",369,2013-11-06,"Restaurants, Nightlife, American (Traditional)...",Lakewood,41.481281,-81.827344,81,4.0,OH
1,rFZPxXnZZuEfO5Cj8SprGw,mhL4vCKi7oW1lVZXk3mTkw,YEgBQZ_UakfJa31NWLODvA,2011-04-16,1,0,2,I can't believe this place hasn't been reviewe...,3.80,"2016, 2014, 2013, 2010, 2012, 2009, 2011, 2017...",483,2007-01-16,"Women's Clothing, Home Decor, Home & Garden, F...",Phoenix,33.457785,-112.066589,18,5.0,AZ
2,DU9CeH0kUfN8LO7V02fxaA,d2ZQRjuizstCTnicysmpMQ,0_TDbuQpN_11ui_zEhzb2A,2012-05-08,1,1,1,This is the best restaurant I've eaten at in t...,4.14,None,7,2011-10-20,"Restaurants, American (New)",Pittsburgh,40.446002,-79.993500,572,4.0,PA
3,dJgLik5M3vvHYHDkIW62qA,JfC7ufsnkzAwNFfoIcAHNw,a9UymHuKxZKMnHXOo3nLKA,2016-05-30,1,1,2,"From what I understand, this is the newest loc...",4.26,"2017, 2016",131,2015-09-14,"Spine Surgeons, Orthopedists, Health & Medical...",Las Vegas,36.054773,-115.271030,33,4.0,NV
4,fi9Pbb8Z4bzzlHaxXGEJTg,yMopev-RPxfEXgqkJOvCGA,R_wlZO-jKFGEVJnSfWISqQ,2014-07-17,7,6,11,"Ok, I love this place. Is that what you want ...",3.83,"2013, 2017, 2012, 2015, 2014, 2016",212,2011-07-01,"Nightlife, Jazz & Blues, Restaurants, Cajun/Cr...",Charlotte,35.214537,-80.821133,558,4.0,NC


In [0]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation='logistic', solver='adam', learning_rate='adaptive', alpha=1e-4, hidden_layer_sizes=(100, 100), random_state=42) 
X = no_text_train
y = no_text_train_y
clf.fit(X, y)
predictions = clf.predict(no_text_test)
accuracy = np.mean(predictions==sample['is_good_rating'])
print(accuracy)

0.7593666666666666


In [0]:
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
X.head()

,authored_by_elite_user,useful,cool,funny,user_average_stars,user_business_avg_star_difference,business_average_stars
0,0,0,0,0,2.00,-2.00,4.0
1,0,0,0,0,4.43,0.93,3.5
2,0,0,0,0,4.09,0.09,4.0
3,0,0,1,0,3.55,0.55,3.0
4,0,1,1,1,3.75,0.25,3.5


In [0]:
y.head()

0    0
1    0
2    1
3    0
4    0
Name: is_good_rating, dtype: int64

In [0]:
preds = pd.read_csv(folder_path+'yelp_sample.csv')
print(preds.head())
preds.is_good_rating = predictions
preds.set_index('review_id', inplace=True)
print(preds.head())
preds.to_csv('preds.csv')
!cp preds.csv /content/gdrive/My\ Drive/Cal/Data\ 144\ Kaggle\ Challenge

                review_id  is_good_rating
0  SiQGQ__poSeG1oKHVem30g               1
1  rFZPxXnZZuEfO5Cj8SprGw               1
2  DU9CeH0kUfN8LO7V02fxaA               1
3  dJgLik5M3vvHYHDkIW62qA               1
4  fi9Pbb8Z4bzzlHaxXGEJTg               1
                        is_good_rating
review_id                             
SiQGQ__poSeG1oKHVem30g               1
rFZPxXnZZuEfO5Cj8SprGw               1
DU9CeH0kUfN8LO7V02fxaA               1
dJgLik5M3vvHYHDkIW62qA               1
fi9Pbb8Z4bzzlHaxXGEJTg               1
